##Felony-based Arrests in New York City during COVID-19 Pandemic in 2020##

[NYPD report](https://www1.nyc.gov/site/nypd/news/p0106a/overall-crime-new-york-city-reaches-record-low-2020) shows that the overall crime in New York City reaches record low in 2020 during the pandemic. However, at the same time, NYPD also stressed that such decrease of crime rate is accompanied by significant upticks in homicides, shootings, burglaries and car thefts. A [New York Times article](https://www.nytimes.com/2020/12/29/nyregion/nyc-2020-crime-covid.html) points out that, the increased crime cases, such as shootings have doubled, and most of them were concentrated in the areas hardest hit by the coronavirus and unemployment. 

According to the ["COVID-19 Data by ZIP Code"](https://www1.nyc.gov/site/doh/covid/covid-19-data-totals.page#zip) released by the health department, Staten Island, Bronx and Queens are the three boroughs that got most seriously hit by the disease (rate per 100,000 people).

In this project, I am going to focus on NYPD's record of **arrests on felony counts** in New York City during the height of pandemic (March - December 2020). Though arrest does not necessarily lead to judiciary crime, NYPD's arrest record provides us a relatively comprehensive account to see the immediate changes in the city during the pandemic.

**My research questions are:** 

*   Did New York City show an increased number of arrests on felony counts during the pandemic in 2020? 
*   What types of arrests on felony counts were mostly recorded?
*   Who were arrested on those felony counts? (demographic characteristics) 
*   Where did those arrests on felony counts usually happen?
*   Does the location of those arrests on felony counts correspondent to the areas most seriously hit by the pandemic?


The datasets I am going to rely on include:

[NYPD Arrests Data (Historic)](https://data.cityofnewyork.us/Public-Safety/NYPD-Arrests-Data-Historic-/8h9b-rp9u) 

[NYPD precinct map](https://data.cityofnewyork.us/Public-Safety/Police-Precincts/78dh-3ptz)

[COVID-19 Data by Date](https://www1.nyc.gov/site/doh/covid/covid-19-data.page#epicurve)


It is also important to note that, the NYPD arrest record inevitably shows establishment institutional bias. 

### Set the Stage ###
Before we proceed the project, we mount our drive which usually contains the datasets we need. We also need to import the modules that are necessary for the project.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

%pip install --upgrade --quiet plotly
import plotly.express as px

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import requests

     |████████████████████████████████| 13.2MB 278kB/s 


### Retrieve NYPD Arrest Data ###
NYC open data allows us to download the [NYPD arrest dataset](https://data.cityofnewyork.us/Public-Safety/NYPD-Arrests-Data-Historic-/8h9b-rp9u). For the purpose of this project, we first need to filter the data according to the time-span needed: March 1 - December 31, 2020, when New York City was mostly seriously hit by the epidemic. 

Then we download the dataset as .csv file and load the dataset for further process.  

In [ ]:
url_2020 = '/content/drive/MyDrive/Colab Notebooks/NYPD_Arrests_Data__2020mar-dec.csv'
df_20 = pd.read_csv(url_2020)
df_20

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
0,210317321,03/01/2020,177.0,SEXUAL ABUSE,116.0,SEX CRIMES,PL 1306504,F,K,66,0,25-44,M,WHITE HISPANIC,986633,167258,40.625769,-73.991417,POINT (-73.99141682199996 40.62576896100006)
1,210302296,03/01/2020,259.0,"CRIMINAL MISCHIEF,UNCLASSIFIED 4",351.0,CRIMINAL MISCHIEF & RELATED OF,PL 1450001,M,M,18,0,25-44,M,WHITE,990319,217877,40.764705,-73.978093,POINT (-73.97809262399994 40.76470460400003)
2,210320615,03/01/2020,101.0,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,Q,102,0,45-64,M,ASIAN / PACIFIC ISLANDER,1031320,193866,40.698677,-73.830249,POINT (-73.83024897799999 40.69867719400003)
3,210324467,03/01/2020,109.0,"ASSAULT 2,1,UNCLASSIFIED",106.0,FELONY ASSAULT,PL 1200512,F,B,47,0,18-24,M,WHITE HISPANIC,1022451,265312,40.894819,-73.861828,POINT (-73.86182751599993 40.89481948400004)
4,210328929,03/01/2020,339.0,"LARCENY,PETIT FROM OPEN AREAS,",341.0,PETIT LARCENY,PL 1552500,M,B,52,0,45-64,M,WHITE HISPANIC,1009690,257590,40.873671,-73.908014,POINT (-73.90801364899994 40.87367103500002)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108765,222450788,12/31/2020,339.0,"LARCENY,PETIT FROM OPEN AREAS,",341.0,PETIT LARCENY,PL 1552500,M,S,121,0,25-44,M,WHITE HISPANIC,940599,167433,40.626142,-74.157253,POINT (-74.15725258299993 40.62614207300004)
108766,222469318,12/31/2020,109.0,"ASSAULT 2,1,UNCLASSIFIED",106.0,FELONY ASSAULT,PL 1200501,F,M,14,1,25-44,M,BLACK,987078,215157,40.757241,-73.989794,POINT (-73.98979363699993 40.757240531000036)
108767,222471322,12/31/2020,439.0,"LARCENY,GRAND FROM OPEN AREAS, UNATTENDED",109.0,GRAND LARCENY,PL 1553501,F,K,73,2,<18,M,BLACK,1010033,187025,40.679988,-73.907044,POINT (-73.907044161 40.67998785000003)
108768,222471330,12/31/2020,792.0,WEAPONS POSSESSION 1 & 2,118.0,DANGEROUS WEAPONS,PL 265031B,F,Q,113,0,<18,M,BLACK,1041498,189225,40.685879,-73.793583,POINT (-73.79358277099993 40.68587876500004)


### NYPD Arrest in 2020 ###

Before we do any comparison or calculation, we can browse through the data to get a first impression. According to the NYPD Arrest record dictionary, we may be interested in the fields of "ARREST_Date", ""OFNS_DESC"(offense description), "LAW_CAT_CD"(category of arrest type, such as felony, misdemeanor), "ARREST_PRECINCT", "AGE_GROUP", "PERP_SEX", and "PERP_RACE".

In [ ]:
# modify the arrest date to yyyy-mm-dd format.

df_20['ARREST_DATE'] = pd.to_datetime(df_20['ARREST_DATE'], format='%m/%d/%Y')
df_20

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
0,210317321,2020-03-01,177.0,SEXUAL ABUSE,116.0,SEX CRIMES,PL 1306504,F,K,66,0,25-44,M,WHITE HISPANIC,986633,167258,40.625769,-73.991417,POINT (-73.99141682199996 40.62576896100006)
1,210302296,2020-03-01,259.0,"CRIMINAL MISCHIEF,UNCLASSIFIED 4",351.0,CRIMINAL MISCHIEF & RELATED OF,PL 1450001,M,M,18,0,25-44,M,WHITE,990319,217877,40.764705,-73.978093,POINT (-73.97809262399994 40.76470460400003)
2,210320615,2020-03-01,101.0,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,Q,102,0,45-64,M,ASIAN / PACIFIC ISLANDER,1031320,193866,40.698677,-73.830249,POINT (-73.83024897799999 40.69867719400003)
3,210324467,2020-03-01,109.0,"ASSAULT 2,1,UNCLASSIFIED",106.0,FELONY ASSAULT,PL 1200512,F,B,47,0,18-24,M,WHITE HISPANIC,1022451,265312,40.894819,-73.861828,POINT (-73.86182751599993 40.89481948400004)
4,210328929,2020-03-01,339.0,"LARCENY,PETIT FROM OPEN AREAS,",341.0,PETIT LARCENY,PL 1552500,M,B,52,0,45-64,M,WHITE HISPANIC,1009690,257590,40.873671,-73.908014,POINT (-73.90801364899994 40.87367103500002)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108765,222450788,2020-12-31,339.0,"LARCENY,PETIT FROM OPEN AREAS,",341.0,PETIT LARCENY,PL 1552500,M,S,121,0,25-44,M,WHITE HISPANIC,940599,167433,40.626142,-74.157253,POINT (-74.15725258299993 40.62614207300004)
108766,222469318,2020-12-31,109.0,"ASSAULT 2,1,UNCLASSIFIED",106.0,FELONY ASSAULT,PL 1200501,F,M,14,1,25-44,M,BLACK,987078,215157,40.757241,-73.989794,POINT (-73.98979363699993 40.757240531000036)
108767,222471322,2020-12-31,439.0,"LARCENY,GRAND FROM OPEN AREAS, UNATTENDED",109.0,GRAND LARCENY,PL 1553501,F,K,73,2,<18,M,BLACK,1010033,187025,40.679988,-73.907044,POINT (-73.907044161 40.67998785000003)
108768,222471330,2020-12-31,792.0,WEAPONS POSSESSION 1 & 2,118.0,DANGEROUS WEAPONS,PL 265031B,F,Q,113,0,<18,M,BLACK,1041498,189225,40.685879,-73.793583,POINT (-73.79358277099993 40.68587876500004)


Now we can get the information about the daily counts of arrests based on different reasons, including felony (LAW_CAT_CD as F), misdemeanor (M), violation (V), and others (I).

In [ ]:
crime_type = df_20.groupby(['LAW_CAT_CD'])
crime_counts_day = crime_type.resample('D', on='ARREST_DATE').size().reset_index(name='day_counts')
crime_counts_day

,LAW_CAT_CD,ARREST_DATE,day_counts
0,F,2020-03-01,171
1,F,2020-03-02,231
2,F,2020-03-03,275
3,F,2020-03-04,346
4,F,2020-03-05,339
...,...,...,...
1216,V,2020-12-27,0
1217,V,2020-12-28,2
1218,V,2020-12-29,3
1219,V,2020-12-30,3


What about we show the daily counts in a line graph?

In [ ]:
fig = px.line(crime_counts_day,x='ARREST_DATE',y='day_counts', color='LAW_CAT_CD',title='Arrest Types During Covid by Day 2020')
fig.show()



The above figure shows the daily counts of arrests on different reasons. We can see the number of arrests on felony and misdemeanor reasons is much higher than the other two.

But this daily count figure is too flucturating to show a clear picture of the trend.

Perhaps a weekly count of arrest records could give us a clearer picture? Let's try. 

In [ ]:
crime_counts_week_20 = crime_type.resample('W', on='ARREST_DATE').size().stack().reset_index(name='week_counts')
crime_counts_week_20

,LAW_CAT_CD,ARREST_DATE,week_counts
0,F,2020-03-01,171
1,F,2020-03-08,1787
2,F,2020-03-15,1704
3,F,2020-03-22,1178
4,F,2020-03-29,815
...,...,...,...
175,V,2020-12-06,5
176,V,2020-12-13,5
177,V,2020-12-20,5
178,V,2020-12-27,4


In [ ]:
fig = px.line(crime_counts_week_20,x='ARREST_DATE',y='week_counts', color='LAW_CAT_CD',title='Arrest Types During Covid by Week 2020')
fig.show()



Hmmmm, this figure of weekly arrest count is much clearer, showing that the number of arrests on felony and misdemeanor counts increased sharply at the beginning of the pandemic, which was March and April in 2020.

But did the number of arrests, especially on felony counts, increased a lot compared to 2019 as the New York Times article claimed? 

### NYPD Arrest in 2019 ###

Now let's take a look at the dataset in 2019 (also March - December).

We can go to the same NYC open data website and retrieve the 2019 NYPD arrest dataset by using the same method, while filtering the data according to the new time-span.

In [ ]:
url_2019 = '/content/drive/MyDrive/Colab Notebooks/NYPD_Arrests_Data__2019mar-dec.csv'
df_19 = pd.read_csv(url_2019)
df_19

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
0,194164531,03/01/2019,177.0,SEXUAL ABUSE,116.0,SEX CRIMES,PL 1306501,F,Q,114,0,25-44,M,BLACK,1007654,219564,40.769306,-73.915508,POINT (-73.91550817999996 40.769306087000075)
1,194153318,03/01/2019,478.0,"THEFT OF SERVICES, UNCLASSIFIE",343.0,OTHER OFFENSES RELATED TO THEF,PL 1651502,M,Q,114,0,25-44,F,BLACK,1007694,219657,40.769561,-73.915363,POINT (-73.91536344699995 40.769561242000066)
2,194168189,03/01/2019,922.0,"TRAFFIC,UNCLASSIFIED MISDEMEAN",348.0,VEHICLE AND TRAFFIC LAWS,VTL05110MU,M,K,76,0,25-44,M,BLACK,982584,184886,40.674154,-74.006007,POINT (-74.00600746599997 40.67415416600005)
3,194129736,03/01/2019,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",113.0,FORGERY,PL 1703000,F,M,25,0,25-44,M,BLACK HISPANIC,1002657,230233,40.798601,-73.933519,POINT (-73.93351914299996 40.79860146500005)
4,194129732,03/01/2019,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",113.0,FORGERY,PL 1703000,F,M,25,0,18-24,M,WHITE HISPANIC,1002657,230233,40.798601,-73.933519,POINT (-73.93351914299996 40.79860146500005)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176829,206884275,12/31/2019,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1600500,F,M,5,0,25-44,M,BLACK,984984,200116,40.715957,-73.997354,POINT (-73.99735367599999 40.71595711700007)
176830,206890198,12/31/2019,268.0,CRIMINAL MIS 2 & 3,121.0,CRIMINAL MISCHIEF & RELATED OF,PL 1450502,F,K,63,0,45-64,F,WHITE,1009348,167036,40.625124,-73.909588,POINT (-73.90958821899994 40.62512438900007)
176831,206892341,12/31/2019,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1600500,F,K,67,0,18-24,M,BLACK HISPANIC,1004940,180941,40.663302,-73.925425,POINT (-73.92542497899997 40.66330200400006)
176832,206890918,12/31/2019,799.0,"PUBLIC SAFETY,UNCLASSIFIED MIS",363.0,OFFENSES AGAINST PUBLIC SAFETY,PL 2700502,M,K,79,2,25-44,M,WHITE HISPANIC,997621,194157,40.699591,-73.951780,POINT (-73.95177985899994 40.699590998000076)


In [ ]:
df_19['ARREST_DATE'] = pd.to_datetime(df_19['ARREST_DATE'], format='%m/%d/%Y')
df_19

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
0,194164531,2019-03-01,177.0,SEXUAL ABUSE,116.0,SEX CRIMES,PL 1306501,F,Q,114,0,25-44,M,BLACK,1007654,219564,40.769306,-73.915508,POINT (-73.91550817999996 40.769306087000075)
1,194153318,2019-03-01,478.0,"THEFT OF SERVICES, UNCLASSIFIE",343.0,OTHER OFFENSES RELATED TO THEF,PL 1651502,M,Q,114,0,25-44,F,BLACK,1007694,219657,40.769561,-73.915363,POINT (-73.91536344699995 40.769561242000066)
2,194168189,2019-03-01,922.0,"TRAFFIC,UNCLASSIFIED MISDEMEAN",348.0,VEHICLE AND TRAFFIC LAWS,VTL05110MU,M,K,76,0,25-44,M,BLACK,982584,184886,40.674154,-74.006007,POINT (-74.00600746599997 40.67415416600005)
3,194129736,2019-03-01,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",113.0,FORGERY,PL 1703000,F,M,25,0,25-44,M,BLACK HISPANIC,1002657,230233,40.798601,-73.933519,POINT (-73.93351914299996 40.79860146500005)
4,194129732,2019-03-01,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",113.0,FORGERY,PL 1703000,F,M,25,0,18-24,M,WHITE HISPANIC,1002657,230233,40.798601,-73.933519,POINT (-73.93351914299996 40.79860146500005)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176829,206884275,2019-12-31,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1600500,F,M,5,0,25-44,M,BLACK,984984,200116,40.715957,-73.997354,POINT (-73.99735367599999 40.71595711700007)
176830,206890198,2019-12-31,268.0,CRIMINAL MIS 2 & 3,121.0,CRIMINAL MISCHIEF & RELATED OF,PL 1450502,F,K,63,0,45-64,F,WHITE,1009348,167036,40.625124,-73.909588,POINT (-73.90958821899994 40.62512438900007)
176831,206892341,2019-12-31,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1600500,F,K,67,0,18-24,M,BLACK HISPANIC,1004940,180941,40.663302,-73.925425,POINT (-73.92542497899997 40.66330200400006)
176832,206890918,2019-12-31,799.0,"PUBLIC SAFETY,UNCLASSIFIED MIS",363.0,OFFENSES AGAINST PUBLIC SAFETY,PL 2700502,M,K,79,2,25-44,M,WHITE HISPANIC,997621,194157,40.699591,-73.951780,POINT (-73.95177985899994 40.699590998000076)


Similarly, we get the weekly count of arrest records in 2019.

In [ ]:
crime_type_19 = df_19.groupby(['LAW_CAT_CD'])
crime_counts_week_19 = crime_type_19.resample('W', on='ARREST_DATE').size().reset_index(name='week_counts_19')
crime_counts_week_19

,LAW_CAT_CD,ARREST_DATE,week_counts_19
0,F,2019-03-03,577
1,F,2019-03-10,1549
2,F,2019-03-17,1862
3,F,2019-03-24,1658
4,F,2019-03-31,1633
...,...,...,...
174,V,2019-12-08,35
175,V,2019-12-15,34
176,V,2019-12-22,18
177,V,2019-12-29,16


### NYPD Arrest 2019-2020 Comparison ###

To answer our question about whether or not felony-reasoned arrests increased in 2020 during the pandemic, we now focus on the number of arrests based on felony suspicion.

Let's get the weekly number of felony-reasoned arrests in 2019 and 2020 respectively.


In [ ]:
crime_counts_week_20_felony = crime_counts_week_20[crime_counts_week_20['LAW_CAT_CD'] == 'F']
crime_counts_week_20_felony

,LAW_CAT_CD,ARREST_DATE,week_counts
0,F,2020-03-01,171
1,F,2020-03-08,1787
2,F,2020-03-15,1704
3,F,2020-03-22,1178
4,F,2020-03-29,815
5,F,2020-04-05,653
6,F,2020-04-12,781
7,F,2020-04-19,877
8,F,2020-04-26,1052
9,F,2020-05-03,1309


In [ ]:
crime_counts_week_19_felony = crime_counts_week_19[crime_counts_week_19['LAW_CAT_CD'] == 'F']
crime_counts_week_19_felony

,LAW_CAT_CD,ARREST_DATE,week_counts_19
0,F,2019-03-03,577
1,F,2019-03-10,1549
2,F,2019-03-17,1862
3,F,2019-03-24,1658
4,F,2019-03-31,1633
5,F,2019-04-07,1810
6,F,2019-04-14,1759
7,F,2019-04-21,1669
8,F,2019-04-28,1573
9,F,2019-05-05,1771


For better viewing, we can merge the weekly felony-reasoned arrest numbers in 2020 and 2019 together.

The default index here can serve as the indicator of number of week, e.g. 1 means Week1. 

In [ ]:
crime_counts_comp_felony = crime_counts_week_20_felony.merge(crime_counts_week_19_felony, how='left', left_index=True, right_index=True)
crime_counts_comp_felony = crime_counts_comp_felony.rename(columns={"week_counts": "week_counts_20"})
crime_counts_comp_felony

,LAW_CAT_CD_x,ARREST_DATE_x,week_counts_20,LAW_CAT_CD_y,ARREST_DATE_y,week_counts_19
0,F,2020-03-01,171,F,2019-03-03,577
1,F,2020-03-08,1787,F,2019-03-10,1549
2,F,2020-03-15,1704,F,2019-03-17,1862
3,F,2020-03-22,1178,F,2019-03-24,1658
4,F,2020-03-29,815,F,2019-03-31,1633
5,F,2020-04-05,653,F,2019-04-07,1810
6,F,2020-04-12,781,F,2019-04-14,1759
7,F,2020-04-19,877,F,2019-04-21,1669
8,F,2020-04-26,1052,F,2019-04-28,1573
9,F,2020-05-03,1309,F,2019-05-05,1771


Now let's see how the number of felony-reasoned arrests changed in 2020 in comparison to 2019.


In [ ]:
crime_counts_comp_felony['change_percentage'] = (crime_counts_comp_felony['week_counts_20'] - crime_counts_comp_felony['week_counts_19'])/crime_counts_comp_felony['week_counts_19']*100
crime_counts_comp_felony


,LAW_CAT_CD_x,ARREST_DATE_x,week_counts_20,LAW_CAT_CD_y,ARREST_DATE_y,week_counts_19,change_percentage
0,F,2020-03-01,171,F,2019-03-03,577,-70.363951
1,F,2020-03-08,1787,F,2019-03-10,1549,15.364751
2,F,2020-03-15,1704,F,2019-03-17,1862,-8.485499
3,F,2020-03-22,1178,F,2019-03-24,1658,-28.950543
4,F,2020-03-29,815,F,2019-03-31,1633,-50.091855
5,F,2020-04-05,653,F,2019-04-07,1810,-63.922652
6,F,2020-04-12,781,F,2019-04-14,1759,-55.599773
7,F,2020-04-19,877,F,2019-04-21,1669,-47.453565
8,F,2020-04-26,1052,F,2019-04-28,1573,-33.121424
9,F,2020-05-03,1309,F,2019-05-05,1771,-26.086957


Hmmm, interestingly, results show that, actually, the weekly felony-reasoned arrests during the pandemic in 2020 most likely decreased (minus percentage) compared to the same period in 2019.

To visualize the 2019-2020 comparison, let's make a line graph.

In [ ]:
crime_counts_comp_felony['time_stamp'] = crime_counts_comp_felony.index
crime_counts_comp_felony

,LAW_CAT_CD_x,ARREST_DATE_x,week_counts_20,LAW_CAT_CD_y,ARREST_DATE_y,week_counts_19,change_percentage,time_stamp
0,F,2020-03-01,171,F,2019-03-03,577,-70.363951,0
1,F,2020-03-08,1787,F,2019-03-10,1549,15.364751,1
2,F,2020-03-15,1704,F,2019-03-17,1862,-8.485499,2
3,F,2020-03-22,1178,F,2019-03-24,1658,-28.950543,3
4,F,2020-03-29,815,F,2019-03-31,1633,-50.091855,4
5,F,2020-04-05,653,F,2019-04-07,1810,-63.922652,5
6,F,2020-04-12,781,F,2019-04-14,1759,-55.599773,6
7,F,2020-04-19,877,F,2019-04-21,1669,-47.453565,7
8,F,2020-04-26,1052,F,2019-04-28,1573,-33.121424,8
9,F,2020-05-03,1309,F,2019-05-05,1771,-26.086957,9


In [ ]:
time_stamp = crime_counts_comp_felony['time_stamp']
week_counts_19 = crime_counts_comp_felony['week_counts_19']
week_counts_20 = crime_counts_comp_felony['week_counts_20']

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=time_stamp, y=week_counts_19, name='weekly felony-reasoned arrests 2019'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=time_stamp, y=week_counts_20, name='weekly felony-reasoned arrests 2020'),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text='Felony-reasoned Arrest Weekly Count 2019-20 Comparison'
)

# Set x-axis title
fig.update_xaxes(title_text="Week")

# Set y-axes titles
fig.update_yaxes(title_text="weekly felony-reasoned arrests 2019", secondary_y=False)
fig.update_yaxes(title_text="weekly felony-reasoned arrests 2020", secondary_y=True)

fig.show()

Now we know that according to NYPD arrest records, the number of felony-reasoned arrests (by weekly counts) in 2020 actually decreased compared to that of 2019.



### Closer Look at NYPD Arrest 2020 ###

Let's take a closer look at the arrest records in 2020.

We want to know, among the arrests based on felony counts, what were the specific reasons.

First, we filter out the felony-reasoned arrest based on "LAW_CAT_CD" as "F".

In [ ]:
df_20_felony = df_20[df_20['LAW_CAT_CD'] == 'F']
df_20_felony

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
0,210317321,2020-03-01,177.0,SEXUAL ABUSE,116.0,SEX CRIMES,PL 1306504,F,K,66,0,25-44,M,WHITE HISPANIC,986633,167258,40.625769,-73.991417,POINT (-73.99141682199996 40.62576896100006)
3,210324467,2020-03-01,109.0,"ASSAULT 2,1,UNCLASSIFIED",106.0,FELONY ASSAULT,PL 1200512,F,B,47,0,18-24,M,WHITE HISPANIC,1022451,265312,40.894819,-73.861828,POINT (-73.86182751599993 40.89481948400004)
5,210357817,2020-03-01,109.0,"ASSAULT 2,1,UNCLASSIFIED",106.0,FELONY ASSAULT,PL 1200512,F,M,19,0,<18,M,BLACK HISPANIC,994297,218680,40.766905,-73.963731,POINT (-73.96373101899997 40.766905006000066)
7,210332448,2020-03-01,109.0,"ASSAULT 2,1,UNCLASSIFIED",106.0,FELONY ASSAULT,PL 1200502,F,M,1,0,25-44,M,BLACK HISPANIC,981760,197923,40.709938,-74.008983,POINT (-74.00898285199997 40.709937533000065)
8,210327480,2020-03-01,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1601501,F,Q,105,0,18-24,M,BLACK,1057767,203993,40.726293,-73.734761,POINT (-73.73476084899994 40.726293088000034)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108762,222471125,2020-12-31,439.0,"LARCENY,GRAND FROM OPEN AREAS, UNATTENDED",109.0,GRAND LARCENY,PL 1553501,F,K,73,2,<18,M,BLACK,1010033,187025,40.679988,-73.907044,POINT (-73.907044161 40.67998785000003)
108764,222467921,2020-12-31,792.0,WEAPONS POSSESSION 1 & 2,118.0,DANGEROUS WEAPONS,PL 265031B,F,K,77,0,<18,M,BLACK,995378,184107,40.672009,-73.959886,POINT (-73.95988567899997 40.67200915000007)
108766,222469318,2020-12-31,109.0,"ASSAULT 2,1,UNCLASSIFIED",106.0,FELONY ASSAULT,PL 1200501,F,M,14,1,25-44,M,BLACK,987078,215157,40.757241,-73.989794,POINT (-73.98979363699993 40.757240531000036)
108767,222471322,2020-12-31,439.0,"LARCENY,GRAND FROM OPEN AREAS, UNATTENDED",109.0,GRAND LARCENY,PL 1553501,F,K,73,2,<18,M,BLACK,1010033,187025,40.679988,-73.907044,POINT (-73.907044161 40.67998785000003)


In [ ]:
df_20_felony['OFNS_DESC'].unique()

array(['SEX CRIMES', 'FELONY ASSAULT', 'ROBBERY',
       'CRIMINAL MISCHIEF & RELATED OF', 'DANGEROUS WEAPONS', 'FORGERY',
       'GRAND LARCENY', 'MISCELLANEOUS PENAL LAW', 'BURGLARY',
       'DANGEROUS DRUGS', 'NYS LAWS-UNCLASSIFIED FELONY',
       'INTOXICATED & IMPAIRED DRIVING', 'INTOXICATED/IMPAIRED DRIVING',
       'MURDER & NON-NEGL. MANSLAUGHTE', 'GRAND LARCENY OF MOTOR VEHICLE',
       'POSSESSION OF STOLEN PROPERTY', 'RAPE', 'THEFT-FRAUD',
       'KIDNAPPING & RELATED OFFENSES', 'VEHICLE AND TRAFFIC LAWS',
       'OTHER STATE LAWS (NON PENAL LA', 'ARSON',
       'PROSTITUTION & RELATED OFFENSES', 'OFF. AGNST PUB ORD SENSBLTY &',
       'HOMICIDE-NEGLIGENT-VEHICLE', 'CHILD ABANDONMENT/NON SUPPORT',
       'HOMICIDE-NEGLIGENT,UNCLASSIFIE', 'OFFENSES AGAINST PUBLIC SAFETY',
       nan, 'ENDAN WELFARE INCOMP', 'GAMBLING', 'KIDNAPPING',
       'KIDNAPPING AND RELATED OFFENSES'], dtype=object)

We can get the counts of felony-reasoned arrests based on different reasons.

From the table below, we can see during pandemic in 2020, most NYPD ***felony-reasoned arrest*** cases were counted on **felony assult**, followed by **miscellaneous penal law**, **robbery** and **burglary**.

In [ ]:
df_20_felony_type = df_20_felony.groupby('OFNS_DESC').size().reset_index(name='counts')
df_20_felony_type.sort_values('counts', ascending=False)

,OFNS_DESC,counts
7,FELONY ASSAULT,11041
19,MISCELLANEOUS PENAL LAW,7175
28,ROBBERY,6187
1,BURGLARY,5155
10,GRAND LARCENY,4993
5,DANGEROUS WEAPONS,4426
4,DANGEROUS DRUGS,4054
3,CRIMINAL MISCHIEF & RELATED OF,2781
8,FORGERY,1303
11,GRAND LARCENY OF MOTOR VEHICLE,1034


What about arrests based on misdemeanor reasons? Let's take a look at the data as well.

In [ ]:
df_20_mis = df_20[df_20['LAW_CAT_CD'] == 'M']
df_20_mis

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
1,210302296,2020-03-01,259.0,"CRIMINAL MISCHIEF,UNCLASSIFIED 4",351.0,CRIMINAL MISCHIEF & RELATED OF,PL 1450001,M,M,18,0,25-44,M,WHITE,990319,217877,40.764705,-73.978093,POINT (-73.97809262399994 40.76470460400003)
2,210320615,2020-03-01,101.0,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,Q,102,0,45-64,M,ASIAN / PACIFIC ISLANDER,1031320,193866,40.698677,-73.830249,POINT (-73.83024897799999 40.69867719400003)
4,210328929,2020-03-01,339.0,"LARCENY,PETIT FROM OPEN AREAS,",341.0,PETIT LARCENY,PL 1552500,M,B,52,0,45-64,M,WHITE HISPANIC,1009690,257590,40.873671,-73.908014,POINT (-73.90801364899994 40.87367103500002)
6,210332280,2020-03-01,101.0,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,Q,111,0,25-44,F,WHITE HISPANIC,1051432,211820,40.747827,-73.757538,POINT (-73.757538492 40.74782657000002)
10,210317319,2020-03-01,114.0,OBSTR BREATH/CIRCUL,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1211100,M,B,48,0,25-44,M,WHITE HISPANIC,1011811,246833,40.844140,-73.900389,POINT (-73.90038861799998 40.84413994500005)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108759,222472996,2020-12-31,681.0,"CHILD, ENDANGERING WELFARE",233.0,SEX CRIMES,PL 2601001,M,M,6,0,25-44,F,ASIAN / PACIFIC ISLANDER,982752,206772,40.734226,-74.005407,POINT (-74.005406689 40.734226128000046)
108760,222433666,2020-12-31,101.0,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,M,14,0,18-24,M,BLACK,985929,214009,40.754090,-73.993941,POINT (-73.99394129799998 40.754089855000075)
108763,222466667,2020-12-31,508.0,"DRUG PARAPHERNALIA, POSSESSE",235.0,DANGEROUS DRUGS,PL 2205003,M,Q,113,0,18-24,M,BLACK,1049137,194504,40.700316,-73.765988,POINT (-73.76598816099995 40.700315647000025)
108765,222450788,2020-12-31,339.0,"LARCENY,PETIT FROM OPEN AREAS,",341.0,PETIT LARCENY,PL 1552500,M,S,121,0,25-44,M,WHITE HISPANIC,940599,167433,40.626142,-74.157253,POINT (-74.15725258299993 40.62614207300004)


In [ ]:
df_20_mis['OFNS_DESC'].unique()

array(['CRIMINAL MISCHIEF & RELATED OF', 'ASSAULT 3 & RELATED OFFENSES',
       'PETIT LARCENY', 'OTHER TRAFFIC INFRACTION',
       'OFFENSES AGAINST PUBLIC ADMINI', 'VEHICLE AND TRAFFIC LAWS',
       'OTHER OFFENSES RELATED TO THEF', 'INTOXICATED & IMPAIRED DRIVING',
       'DANGEROUS DRUGS', 'DANGEROUS WEAPONS',
       'OFF. AGNST PUB ORD SENSBLTY &', 'CRIMINAL TRESPASS',
       'OTHER STATE LAWS (NON PENAL LA', 'UNAUTHORIZED USE OF A VEHICLE',
       'THEFT OF SERVICES', 'OFFENSES AGAINST THE PERSON',
       'POSSESSION OF STOLEN PROPERTY', 'SEX CRIMES', "BURGLAR'S TOOLS",
       'OFFENSES INVOLVING FRAUD', 'OTHER STATE LAWS (NON PENAL LAW)',
       'FRAUDS', 'PROSTITUTION & RELATED OFFENSES',
       'FRAUDULENT ACCOSTING', 'JOSTLING', 'GAMBLING',
       'ALCOHOLIC BEVERAGE CONTROL LAW', 'OFFENSES RELATED TO CHILDREN',
       'ESCAPE 3', 'ADMINISTRATIVE CODE', nan,
       'OFFENSES AGAINST PUBLIC SAFETY', 'ENDAN WELFARE INCOMP',
       'AGRICULTURE & MRKTS LAW-UNCLASSIFIED', 'ANTICI

From this table, we can see during pandemic in 2020, most NYPD ***misdemeanor-reasoned arrest*** cases were about **assult (degree 3) and related offenses**, **petit larceny** and **dangerous drugs**.

In [ ]:
df_20_mis_type = df_20_mis.groupby('OFNS_DESC').size().reset_index(name='counts')
df_20_mis_type.sort_values('counts', ascending=False)


,OFNS_DESC,counts
4,ASSAULT 3 & RELATED OFFENSES,19011
27,PETIT LARCENY,8402
8,DANGEROUS DRUGS,4152
18,OFFENSES AGAINST PUBLIC ADMINI,3756
6,CRIMINAL MISCHIEF & RELATED OF,3489
33,VEHICLE AND TRAFFIC LAWS,2524
17,OFF. AGNST PUB ORD SENSBLTY &,2441
30,SEX CRIMES,1782
15,INTOXICATED & IMPAIRED DRIVING,1565
9,DANGEROUS WEAPONS,1179


We also want to know the demographic characteristics of those being arrested on felony counts. Unsurprisingly, black is the race that tops in the counts. Historically, police force is known for racial profiling.

In [ ]:
df_20_felony_race = df_20_felony.groupby(['PERP_RACE']).size()
df_20_felony_race.sort_values(ascending=False)

PERP_RACE
BLACK                             27823
WHITE HISPANIC                    12086
WHITE                              5006
BLACK HISPANIC                     4487
ASIAN / PACIFIC ISLANDER           2451
UNKNOWN                             130
AMERICAN INDIAN/ALASKAN NATIVE      104
dtype: int64

And the demographic characteristics of misdemeanor-based arrests is not much different, with black people tops in the counts.

In [ ]:
df_20_mis_race = df_20_mis.groupby(['PERP_RACE']).size()
df_20_mis_race.sort_values(ascending=False)

PERP_RACE
BLACK                             25247
WHITE HISPANIC                    14320
WHITE                              6966
BLACK HISPANIC                     4855
ASIAN / PACIFIC ISLANDER           3293
UNKNOWN                             290
AMERICAN INDIAN/ALASKAN NATIVE      187
dtype: int64

What about perpetrators' sex? Again, without much surprise, the number of male is significantly higher than female. The situation is as the same as in misdemeanor-based arrests.

In [ ]:
df_20_felony_sex = df_20_felony.groupby(['PERP_SEX']).size()
df_20_felony_sex.sort_values(ascending=False)

PERP_SEX
M    44653
F     7434
dtype: int64

In [ ]:
df_20_mis_sex = df_20_mis.groupby(['PERP_SEX']).size()
df_20_mis_sex.sort_values(ascending=False)

PERP_SEX
M    44611
F    10547
dtype: int64

What about perpetrators' age? According to the results, for felony-reasoned arrests, most perpetrators fall into 25-44 age group. For misdemeanor-reasoned arrests, the same result applies.

In [ ]:
df_20_felony_age = df_20_felony.groupby(['AGE_GROUP']).size()
df_20_felony_age.sort_values(ascending=False)


AGE_GROUP
25-44    27910
18-24    11557
45-64     9112
<18       3002
65+        506
dtype: int64

In [ ]:
df_20_mis_age = df_20_mis.groupby(['AGE_GROUP']).size()
df_20_mis_age.sort_values(ascending=False)

AGE_GROUP
25-44    31351
45-64    11336
18-24    10380
<18       1268
65+        823
dtype: int64

We can see that those being arrested based on felony and misdemeanor reasons were mostly black male aged 25-44.

### Arrest and Region ###

Now let's take a look at where did felony-reasoned arrests mostly take place.

In [ ]:
df_20_felony_boro = df_20_felony.groupby(['ARREST_BORO']).size().reset_index(name='counts')
df_20_felony_boro.sort_values('counts', ascending=False)


,ARREST_BORO,counts
1,K,15882
2,M,11568
3,Q,11193
0,B,11110
4,S,2334


From the above table, we can see those felony-based arrests took place mostly in Brooklyn (K), followed by Manhattan (M), Queens (Q), Bronx (B), and Staten Island (S).

### Arrest and Covid-19 ### 

Let's see if the number of felony-reasoned arrest corresponded to the level of seriousness caused by Covid-19 in different boroughs

The [data of Covid-19](https://www1.nyc.gov/site/doh/covid/covid-19-data.page#epicurve) could be retrieved via the nyc.gov. The data shows the situation of Covid-infection day-by-day in different boroughs.

In [ ]:
df_covid_date = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/covid-data-by-day.csv')
df_covid_date

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,BX_PROBABLE_DEATH_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BX_HOSPITALIZED_COUNT_7DAY_AVG,BX_DEATH_COUNT_7DAY_AVG,BX_ALL_DEATH_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_PROBABLE_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BK_PROBABLE_DEATH_COUNT,BK_CASE_COUNT_7DAY_AVG,BK_ALL_CASE_COUNT_7DAY_AVG,BK_HOSPITALIZED_COUNT_7DAY_AVG,BK_DEATH_COUNT_7DAY_AVG,BK_ALL_DEATH_COUNT_7DAY_AVG,MN_CASE_COUNT,MN_PROBABLE_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,MN_PROBABLE_DEATH_COUNT,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,MN_HOSPITALIZED_COUNT_7DAY_AVG,MN_DEATH_COUNT_7DAY_AVG,MN_ALL_DEATH_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,QN_PROBABLE_DEATH_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,QN_ALL_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,SI_ALL_DEATH_COUNT_7DAY_AVG,INCOMPLETE
0,02/29/2020,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,03/01/2020,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,03/02/2020,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,03/03/2020,1,0,7,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,03/04/2020,5,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,04/15/2021,1934,626,196,31,7,2031,2758,176,41,45,342,107,42,7,0,312,444,35,6,7,611,186,54,9,0,669,892,51,15,15,270,90,29,4,1,269,378,25,6,6,538,185,58,7,1,599,808,53,12,13,173,53,13,4,0,181,234,13,2,2,11000
412,04/16/2021,1749,625,170,22,9,1932,2636,173,37,41,245,130,30,2,0,296,429,34,5,6,580,185,56,9,1,639,855,52,13,13,215,95,21,2,0,254,358,24,5,5,531,163,49,7,1,572,771,52,11,12,178,45,14,2,0,171,221,12,2,2,11000
413,04/17/2021,1198,463,138,24,7,1865,2542,168,33,38,193,73,23,3,1,291,414,32,5,5,384,162,48,12,0,615,826,51,12,13,123,71,13,5,0,244,346,22,5,5,387,111,45,3,0,548,738,51,9,10,111,40,9,1,0,167,214,11,2,2,11000
414,04/18/2021,951,471,76,21,10,1819,2468,158,29,35,137,87,20,1,1,286,403,31,4,5,333,124,25,11,0,599,799,49,11,11,105,66,5,4,0,239,337,21,4,5,283,138,20,3,0,533,716,47,7,8,93,47,6,2,0,162,209,11,2,2,11000


In [ ]:
df_covid_date['date_of_interest'] = pd.to_datetime(df_covid_date['date_of_interest'], format='%m/%d/%Y')
df_covid_date

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,BX_PROBABLE_DEATH_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BX_HOSPITALIZED_COUNT_7DAY_AVG,BX_DEATH_COUNT_7DAY_AVG,BX_ALL_DEATH_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_PROBABLE_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BK_PROBABLE_DEATH_COUNT,BK_CASE_COUNT_7DAY_AVG,BK_ALL_CASE_COUNT_7DAY_AVG,BK_HOSPITALIZED_COUNT_7DAY_AVG,BK_DEATH_COUNT_7DAY_AVG,BK_ALL_DEATH_COUNT_7DAY_AVG,MN_CASE_COUNT,MN_PROBABLE_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,MN_PROBABLE_DEATH_COUNT,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,MN_HOSPITALIZED_COUNT_7DAY_AVG,MN_DEATH_COUNT_7DAY_AVG,MN_ALL_DEATH_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,QN_PROBABLE_DEATH_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,QN_ALL_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,SI_ALL_DEATH_COUNT_7DAY_AVG,INCOMPLETE
0,2020-02-29,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2020-03-01,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2020-03-02,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2020-03-03,1,0,7,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2020-03-04,5,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,2021-04-15,1934,626,196,31,7,2031,2758,176,41,45,342,107,42,7,0,312,444,35,6,7,611,186,54,9,0,669,892,51,15,15,270,90,29,4,1,269,378,25,6,6,538,185,58,7,1,599,808,53,12,13,173,53,13,4,0,181,234,13,2,2,11000
412,2021-04-16,1749,625,170,22,9,1932,2636,173,37,41,245,130,30,2,0,296,429,34,5,6,580,185,56,9,1,639,855,52,13,13,215,95,21,2,0,254,358,24,5,5,531,163,49,7,1,572,771,52,11,12,178,45,14,2,0,171,221,12,2,2,11000
413,2021-04-17,1198,463,138,24,7,1865,2542,168,33,38,193,73,23,3,1,291,414,32,5,5,384,162,48,12,0,615,826,51,12,13,123,71,13,5,0,244,346,22,5,5,387,111,45,3,0,548,738,51,9,10,111,40,9,1,0,167,214,11,2,2,11000
414,2021-04-18,951,471,76,21,10,1819,2468,158,29,35,137,87,20,1,1,286,403,31,4,5,333,124,25,11,0,599,799,49,11,11,105,66,5,4,0,239,337,21,4,5,283,138,20,3,0,533,716,47,7,8,93,47,6,2,0,162,209,11,2,2,11000


In order to correspond to the NYPD arrest dataset, here for the Covid dataset we also select the date range within March and December 2020

In [ ]:
df_covid_date_2020 = df_covid_date[(df_covid_date['date_of_interest'] > '2020-2-29') & (df_covid_date['date_of_interest'] < '2021-1-1')] 
df_covid_date_2020

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,BX_PROBABLE_DEATH_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BX_HOSPITALIZED_COUNT_7DAY_AVG,BX_DEATH_COUNT_7DAY_AVG,BX_ALL_DEATH_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_PROBABLE_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BK_PROBABLE_DEATH_COUNT,BK_CASE_COUNT_7DAY_AVG,BK_ALL_CASE_COUNT_7DAY_AVG,BK_HOSPITALIZED_COUNT_7DAY_AVG,BK_DEATH_COUNT_7DAY_AVG,BK_ALL_DEATH_COUNT_7DAY_AVG,MN_CASE_COUNT,MN_PROBABLE_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,MN_PROBABLE_DEATH_COUNT,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,MN_HOSPITALIZED_COUNT_7DAY_AVG,MN_DEATH_COUNT_7DAY_AVG,MN_ALL_DEATH_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,QN_PROBABLE_DEATH_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,QN_ALL_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,SI_ALL_DEATH_COUNT_7DAY_AVG,INCOMPLETE
1,2020-03-01,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2020-03-02,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2020-03-03,1,0,7,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2020-03-04,5,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2020-03-05,3,0,14,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,3,0,3,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,2020-12-27,2886,933,248,42,3,3241,4101,283,41,44,480,170,38,4,0,537,687,49,6,6,912,230,91,14,1,986,1234,94,11,13,314,130,32,9,1,428,542,45,7,8,869,322,69,6,1,972,1243,78,11,11,311,81,18,9,0,319,395,18,6,6,0
303,2020-12-28,5378,1209,352,44,3,3389,4272,286,42,44,1015,209,68,8,1,579,734,49,7,7,1596,350,105,13,0,1027,1279,95,12,13,671,164,56,5,0,439,555,44,7,7,1534,396,103,11,2,1007,1289,79,10,11,562,90,20,7,0,338,415,19,6,6,0
304,2020-12-29,5207,1012,320,50,4,3547,4438,292,41,44,994,198,61,9,0,617,771,50,7,7,1412,285,84,15,2,1049,1304,95,12,14,610,126,58,8,1,448,564,45,7,7,1605,320,99,9,1,1063,1352,83,10,10,585,82,18,9,0,369,447,18,6,6,0
305,2020-12-30,4976,1063,319,40,4,3709,4618,294,41,44,968,185,68,4,0,664,820,53,6,7,1394,299,96,16,2,1092,1345,92,13,14,653,156,42,5,0,467,589,45,6,7,1497,331,94,11,1,1108,1405,86,10,11,463,92,19,4,1,378,458,17,6,6,0


Then we focus on the Covid case numbers in different boroughs.

In [ ]:
df_covid_date_2020_boro = df_covid_date_2020[['date_of_interest', 'BX_CASE_COUNT', 'BK_CASE_COUNT', 'MN_CASE_COUNT', 'QN_CASE_COUNT', 'SI_CASE_COUNT']]
df_covid_date_2020_boro

,date_of_interest,BX_CASE_COUNT,BK_CASE_COUNT,MN_CASE_COUNT,QN_CASE_COUNT,SI_CASE_COUNT
1,2020-03-01,0,0,0,0,0
2,2020-03-02,0,0,0,0,0
3,2020-03-03,0,0,0,1,0
4,2020-03-04,0,1,2,2,0
5,2020-03-05,0,3,0,0,0
...,...,...,...,...,...,...
302,2020-12-27,480,912,314,869,311
303,2020-12-28,1015,1596,671,1534,562
304,2020-12-29,994,1412,610,1605,585
305,2020-12-30,968,1394,653,1497,463


Then we sum the Covid infection case numbers in different boroughs in year 2020


In [ ]:
print(sum(df_covid_date_2020_boro['BX_CASE_COUNT']))
print(sum(df_covid_date_2020_boro['BK_CASE_COUNT']))
print(sum(df_covid_date_2020_boro['MN_CASE_COUNT']))
print(sum(df_covid_date_2020_boro['QN_CASE_COUNT']))
print(sum(df_covid_date_2020_boro['SI_CASE_COUNT']))

80385
114977
55134
116851
32610


Since the populations in different boroughs are different, we cannot simply use the total Covid case number to do comparison. Therefore, we calculate the Covid infection rate in different boroughs
We use the data from the [Census department](https://www.census.gov/quickfacts/fact/table/newyorkcitynewyork,bronxcountybronxboroughnewyork,kingscountybrooklynboroughnewyork,newyorkcountymanhattanboroughnewyork,queenscountyqueensboroughnewyork,richmondcountystatenislandboroughnewyork/PST045219) to calculate the rate (cases per 100,000 people).

In [ ]:
print(round((80385/1418207) * 100000)) #Bronx population
print(round((114977/2559903) * 100000)) #Brooklyn population
print(round((55134/1628706) * 100000)) #Manhattan population
print(round((116851/2253858) * 100000)) #Queens population
print(round((32610/476143) * 100000)) #StatenIsland population

5668
4491
3385
5184
6849


In [ ]:
# make a dataframe of Covid infection rate in differnet boroughs
d = {'Boro': ['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'StatenIsland'], 'Covid Rate': [5668, 4491, 3385, 5184, 6849]}
df_covid_boro = pd.DataFrame(data=d)
df_covid_boro

,Boro,Covid Rate
0,Bronx,5668
1,Brooklyn,4491
2,Manhattan,3385
3,Queens,5184
4,StatenIsland,6849


In order to have a cross-comparison with the Covid infection rate, now we also need to calculate the arrest rate in different boroughs. We focus on the felony-based arrest per 100,000 population.

But before we proceed to calculation, some modification needs to be done on the NYPD data, by transforming the short-formed borough name into their fully spell-out form, for easier comprehension.

In [ ]:
def boro(row):
  if row.ARREST_BORO == 'K':
    return 'Brooklyn'
  elif row.ARREST_BORO == 'M':
    return 'Manhattan'
  elif row.ARREST_BORO == 'Q':
    return 'Queens'
  elif row.ARREST_BORO == 'B':
    return 'Bronx'
  elif row.ARREST_BORO == 'S':
    return 'StatenIsland'

In [ ]:
# add a new column to the NYPD data which contains the full names of the boroughs

df_20_felony_boro['Boro'] = df_20_felony_boro.apply(boro, axis=1)
df_20_felony_boro

,ARREST_BORO,counts,Boro
0,B,11110,Bronx
1,K,15882,Brooklyn
2,M,11568,Manhattan
3,Q,11193,Queens
4,S,2334,StatenIsland


In [ ]:
# calculate the felony-reasoned arrest rate (arrest per 100,000 population) in different boroughs based on the 2020 NYPD arrest dataset

print(round((11110/1418207) * 100000)) #Bronx population
print(round((15882/2559903) * 100000)) #Brooklyn population
print(round((11568/1628706) * 100000)) #Manhattan population
print(round((11193/2253858) * 100000)) #Queens population
print(round((2334/476143) * 100000)) #StatenIsland population

783
620
710
497
490


In [ ]:
d = {'Boro': ['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'StatenIsland'], 'Arrest Rate': [783,620,710,497,490]}
df_arrest_boro = pd.DataFrame(data=d)
df_arrest_boro

,Boro,Arrest Rate
0,Bronx,783
1,Brooklyn,620
2,Manhattan,710
3,Queens,497
4,StatenIsland,490


Now the felony-reasoned arrest dataset from NYPD will be merged with the Covid case by borough dataset

In [ ]:
df_20_felony_covid_boro = df_arrest_boro.merge(df_covid_boro, left_on='Boro', right_on='Boro')
df_20_felony_covid_boro

,Boro,Arrest Rate,Covid Rate
0,Bronx,783,5668
1,Brooklyn,620,4491
2,Manhattan,710,3385
3,Queens,497,5184
4,StatenIsland,490,6849


From the table above, we can see Bronx has the highest felony-based arrest rate, while in terms of Covid infection rate, Staten Island is higher than other boroughs.

But for a clearer viewing of the order of arrest rates and Covid infection rates according to NYC boroughs, let's make bar charts.

In [ ]:
boro = df_20_felony_covid_boro['Boro']
arrest = df_20_felony_covid_boro['Arrest Rate']
covid = df_20_felony_covid_boro['Covid Rate']


fig = go.Figure()
fig.add_trace(go.Bar(x=boro, y=arrest, marker_color='rgb(0,102,204)'))

fig.update_layout(title_text='Felony-reasoned Arrest Rate in NYC Boroughs 2020')
fig.show()


From this bar chart, we can see the NYPD felony-reasoned arrest rate in Bronx was the highest during the pandemic period in 2020, followed by Manhattan and Brooklyn.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=boro, y=covid, marker_color='rgb(204,0,102)'))

fig.update_layout(title_text='Covid Infection Rate in NYC Boroughs 2020')
fig.show()


This bar chart shows that, the Covid infection rate in Staten Island was the highest in 2020 among other boroughs, folowed by Bronx and Queens.


From these two bar charts, we can see that **the trend of felony-reasoned arrest DOES NOT always correspondent to the serious level of Covid-infection of that borough**. While Staten Island had the highest Covid infection rate during 2020, the felony-reasoned NYPD arrest rate in that borough was actually the lowest among the other boroughs.

### Mapping Arrests ###

Now we want to know where those felony-reasoned arrests took place most, at a more granular level. Let's see the counts of felony-reasoned arrests by precinct.

In [ ]:
df_20_felony_precinct = df_20_felony.groupby(['ARREST_PRECINCT']).size().reset_index(name='counts')
df_20_felony_precinct.sort_values('counts', ascending=False)

,ARREST_PRECINCT,counts
46,75,1871
22,40,1470
45,73,1356
26,44,1254
60,103,1230
...,...,...
56,94,248
76,123,192
49,78,183
68,111,165


With only the precinct numbers, we have no idea where did all those felony-reasoned arrests happen. A map (something like a heatmap) will certainly give a better view.

In [ ]:
geojson = 'https://data.cityofnewyork.us/api/geospatial/78dh-3ptz?method=export&format=GeoJSON'
response = requests.get(geojson)
response.json()

{'features': [{'geometry': {'coordinates': [[[[-74.0438776157395,
        40.69018767637665],
       [-74.0435059601254, 40.68968735963635],
       [-74.04273533826982, 40.69005019142044],
       [-74.04278433380006, 40.69012097669115],
       [-74.04270428426766, 40.690155204644306],
       [-74.04255372037308, 40.6899627592896],
       [-74.0426392937119, 40.68992817641333],
       [-74.0426938081918, 40.689997259107216],
       [-74.04346752310265, 40.68963699010347],
       [-74.04351637245855, 40.68919103374234],
       [-74.04364078627412, 40.68876655957014],
       [-74.04397458556184, 40.68858240705591],
       [-74.0443852177728, 40.688516178402686],
       [-74.04478399040363, 40.68859566011588],
       [-74.04627539003668, 40.689327425896714],
       [-74.04680284898575, 40.68995325626601],
       [-74.04747651462345, 40.68961136999828],
       [-74.04772962763064, 40.68991531846602],
       [-74.04758571924786, 40.68998250682616],
       [-74.04743126123475, 40.689803889968

In [ ]:
fig = px.choropleth_mapbox(df_20_felony_precinct,
                           geojson=geojson,
                           locations='ARREST_PRECINCT',
                           featureidkey='properties.precinct',
                           color='counts',
                           color_continuous_scale='oranges',
                           center = {'lat': 40.73, 'lon': -73.98},
                           hover_data=['ARREST_PRECINCT'],
                           zoom=9,
                           mapbox_style='carto-positron', title='Felony-reasoned Arrests by NY Precinct 2020')

fig.update_layout(height=650)
fig.show()

This map (a heat-map like) revealss a clear picture of where exactly the felony-reasoned arrests happened the most during the Covid pandemic in 2020. Precinct 75, located in Brooklyn, shows significantly darker color compared to other regions, meaning that the number of felony-reasoned arrests topped in that area. According to a [BKLYNER article](https://bklyner.com/what-will-it-take-to-fix-the-citys-most-troubled-police-precinct/), Precinct 75 (covers East New York and Cypress Hills) is historically a low-income region with noticeably high crime rate. "The NYPD’s 75th Precinct had the most gun violence incidents in the city last year", stated in the article. Ironically, this is also a precinct that is historically known for serious police corruption. And residents and community workers have been complaining about the problem of over-policing which, instead of curbs violence, but incites them. There is even a documentary [The Seven Five](https://en.wikipedia.org/wiki/The_Seven_Five) which depicts the exact issue of police misconduct. This complicated police-community dynamic worthes further study for policymakers. 

Areas west to Precinct 75 - Precincts 73 and 67 also show relatively dark organge color. 

Another area shows dark orange color is Precinct 40, in South Bronx, as well as its neighboring Precincts 44 and 46. 